 # YOLO Container Classificatie
 Dit notebook traint een YOLOv8/YOLOv9 classificatiemodel om te bepalen:
 **"Staat er een container op de foto?"**

 We gebruiken hiervoor de dataset die al bestaat:

 temp_data/
   train/
     container/
     geen_container/
   val/
     container/
     geen_container/


In [26]:
from ultralytics import YOLO
import os
import numpy as np
import matplotlib.pyplot as plt

In [9]:
import os

print(os.path.exists("C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/train/container"))
print(os.path.exists("C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/train/geen_container"))
print(os.path.exists("C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/val/container"))
print(os.path.exists("C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/val/geen_container"))


True
True
True
True


In [12]:
import yaml

with open("yolo_data.yaml", "r") as f:
    data = yaml.safe_load(f)

print(data)


{'train': 'C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/train', 'val': 'C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/val', 'nc': 2, 'names': ['container', 'geen_container']}


In [ ]:
from ultralytics import YOLO

# Laad YOLOv8-classification model
model = YOLO("yolov8n-cls.pt")

# Geef hier **alleen de root van de dataset**
data_root = r"C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data"

results = model.train(
    data=data_root,
    epochs=4,
    batch=32,
    imgsz=224,
    lr0=0.001,  # let op: gebruik lr0, niet lr
    name="container_classification"
)

Ultralytics 8.3.229  Python-3.11.9 torch-2.9.1+cpu CPU (12th Gen Intel Core i7-12650H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=4, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=container_classification3, nbs=64, nms=False, opset=None, optimize=F

In [33]:
from ultralytics import YOLO
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

# Laad je model
model = YOLO("runs/classify/container_classification3/weights/best.pt")  # of jouw .pt file

# Directory van validation images
val_dir = "C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/val"
class_names = ["container", "geen_container"]

# Maak lijsten van images en labels
val_images = []
y_true = []

for i, cls in enumerate(class_names):
    cls_dir = os.path.join(val_dir, cls)
    for f in os.listdir(cls_dir):
        val_images.append(os.path.join(cls_dir, f))
        y_true.append(i)

y_true = np.array(y_true)

# Predictions
y_pred = []
for img_path in val_images:
    result = model.predict(img_path, verbose=False)[0]  # klas voorspelling
    pred_label = result.probs.top1  # hoogste probability class index
    y_pred.append(pred_label)

y_pred = np.array(y_pred)

# Metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
accuracy = (y_true == y_pred).mean()
cm = confusion_matrix(y_true, y_pred)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")
print("Confusion matrix:\n", cm)

# Plot confusion matrix
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


Accuracy: 0.896
Precision: 0.898
Recall: 0.931
F1-score: 0.914
Confusion matrix:
 [[301  55]
 [ 36 482]]


<Figure size 600x500 with 2 Axes>

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os
import random
import numpy as np
from tensorflow.keras.preprocessing import image

# Laad YOLO-classify model
model_path = "runs/classify/container_classification3/weights/best.pt"
model = YOLO(model_path)

# Validation folder en class names
val_dir = "C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/val"
class_names = ["container", "geen_container"]

# Lijst met alle validation images
val_images = []
for i, cls in enumerate(class_names):
    cls_dir = os.path.join(val_dir, cls)
    for f in os.listdir(cls_dir):
        val_images.append(os.path.join(cls_dir, f))

# Random subset kiezen voor handmatige check
random.seed(42)
check_images = random.sample(val_images, 20)  # kies bv. 20 afbeeldingen

# Plotten met voorspellingen
plt.figure(figsize=(15, 10))
for idx, img_path in enumerate(check_images):
    # Laad afbeelding
    img_loaded = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img_loaded)

    # YOLO voorspelling
    result = model.predict(img_path, verbose=False)[0]
    pred_label_idx = result.top1  # hoogste waarschijnlijkheid
    pred_conf = float(result.top1conf)  # confidence
    pred_label = class_names[pred_label_idx]

    # Plotten
    plt.subplot(4, 5, idx+1)  # 4x5 grid voor 20 images
    plt.imshow(img_loaded)
    plt.title(f"{pred_label}\n{pred_conf:.2f}")
    plt.axis('off')

plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'tensorflow'